In [ ]:
#| default_exp datasources.connectors
%load_ext autoreload
%autoreload 2

import sys,os
from pathlib import Path

In [ ]:
# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
os.chdir(Path.cwd().parent / 'extracao')

# Conectores
> Módulo para encapsular a criação de conectores de banco com o intuito de serem usados por composição nas classes de extração de dados

In [ ]:
#| export 
from concurrent.futures import ThreadPoolExecutor
from dataclasses import dataclass

import pyodbc
from pymongo import MongoClient

# from motor.motor_asyncio import AsyncIOMotorClient

In [ ]:
# | export
@dataclass
class DBConnector:
    def connect(self):
        raise NotImplementedError

    def get_parallel_connections(self, n):
        # This method returns a list of n pyodbc connection objects in parallel
        with ThreadPoolExecutor(max_workers=n) as executor:
            futures = [executor.submit(self.connect) for _ in range(n)]
            return [future.result() for future in futures]


@dataclass
class SQLServer(DBConnector):
    sql_params: dict

    def connect(self):
        # This method returns a pyodbc connection object according to the platform
        connection_string = f"""Driver={self.sql_params.get('driver')};
                                Server={self.sql_params.get('server')};
                                Database={self.sql_params.get('database')};
                                Encrypt=no;"""
        if self.sql_params.get("trusted_conn"):
            connection_string += f"""MultipleActiveResultSets={self.sql_params.get('mult_results')};
                                  Trusted_Connection=yes;"""
        else:
            connection_string += f"""UID={self.sql_params.get('username')};
                                  PWD={self.sql_params.get('password')};
                                  Trusted_Connection=no;"""
        try:
            return pyodbc.connect(
                connection_string, timeout=self.sql_params.get("timeout", 10000)
            )
        except pyodbc.OperationalError as e:
            raise ConnectionError(
                "Não foi possível abrir uma conexão com o SQL Server. Esta conexão somente funciona da rede cabeada!"
            ) from e


@dataclass
class MongoDB(DBConnector):
    mongo_uri: str

    def connect(self):
        # This method returns a connected AsyncIOMotorClient object
        return MongoClient(self.mongo_uri)

In [ ]:
#|eval:false
from dotenv import find_dotenv, load_dotenv
from extracao.constants import SQL_RADCOM, SQL_STEL
from fastcore.test import test_eq

In [ ]:
#|eval:false
load_dotenv(find_dotenv())

In [ ]:
#|eval:false
def test_connection():
    conn = SQLServer(os.environ['SQL_PARAMS']).connect()
    cursor = conn.cursor()
    for query in (SQL_RADCOM,SQL_STEL):
        cursor.execute(query)
        test_eq(type(cursor.fetchone()), pyodbc.Row)
test_connection()